## 실습 12: FactScore 기반 사실 검증 전체 흐름

이 실습에서는 모델이 생성한 문장 속 사실 단위의 진실 여부를 검증하고,  
이를 정량화한 FactScore 를 계산한다.

### 1) Claim Extraction (사실 단위 분리)
- 하나의 문장에는 여러 개의 사실 요소가 포함될 수 있음
- LLM을 이용해 입력 텍스트를 Atomic Claim(독립적으로 검증 가능한 최소 단위) 으로 분리한다
- 출력 형식은 JSON 배열 형태의 문자열 리스트

In [ ]:
from openai import OpenAI
import requests, json, re
import pandas as pd

OPENAI_API_KEY = "my-key"
SERPER_API_KEY = "my-key"

client = OpenAI(api_key=OPENAI_API_KEY)

### 2) External Knowledge Verification (외부 지식 기반 검증)
- 분리된 각 Claim 은 검색을 통해 외부 근거를 수집한다
- 본 실습에서는 Serper.dev (Google 기반 검색 API) 를 통해:
  - Claim 과 관련된 snippet (근거 문장) 을 수집하고
  - 해당 정보의 출처 URL을 함께 반환한다

In [ ]:
# Claim Extraction

def extract_claims(text: str):
    prompt = f"""
다음 문단을 사실 단위(atomic claim)로 분리하세요.
각 claim은 독립적으로 사실 여부를 판단할 수 있는 최소 단위여야 합니다.
아래 JSON 형식만 출력하세요.

[
  "Claim 1",
  "Claim 2",
  "Claim 3"
]

문단:
{text}
""".strip()

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    raw = resp.choices[0].message.content.strip()

    try:
        parsed = json.loads(raw)
        if isinstance(parsed, dict) and "claims" in parsed:
            parsed = parsed["claims"]
        claims = parsed if isinstance(parsed, list) else []
    except Exception:
        claims = re.findall(r'"([^"]+)"', raw)

    clean = []
    for c in claims:
        c = str(c).strip()
        if not c or c in {"[", "]", "{", "}", ",", ":", "json"}:
            continue
        clean.append(c)
    return clean

### 3) LLM Judgment (사실 여부 판단)
- 수집된 snippet을 LLM 에 전달하여
  - Claim 이 True/False 인지 판정하고
  - 근거를 한 문장으로 요약한다

In [3]:
# (3) Fact Verification

def verify_claim(claim: str, top_k: int = 3, max_snippet_chars: int = 800):
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
    payload = {"q": claim, "num": top_k}

    try:
        r = requests.post("https://google.serper.dev/search", headers=headers, json=payload, timeout=15)
        r.raise_for_status()
        data = r.json()
    except Exception as e:
        return {"claim": claim, "result": "False", "evidence": f"(search failed: {e})", "sources": "-"}

    snippets, urls = [], []
    for item in (data.get("organic") or [])[:top_k]:
        snip = item.get("snippet", "")
        link = item.get("link", "")
        if snip:
            snippets.append(snip)
        if link:
            urls.append(link)

    snippet_text = " ".join(snippets)[:max_snippet_chars]
    source_text = "\n".join(f"- {u}" for u in urls) if urls else "- (no hits)"

    prompt = f"""
아래는 사실 검증을 위한 검색 결과입니다.
주어진 Claim이 사실인지 판단하고, 판단 근거를 한 문장으로 요약하세요.
마지막 줄에는 반드시 다음 중 하나만 작성하세요: "판단: True" 또는 "판단: False"

[Claim]
{claim}

[검색 결과]
{snippet_text}
""".strip()

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )

    content = resp.choices[0].message.content.strip()
    is_true = "판단: true" in content.lower() or content.strip().lower().endswith("판단: true")
    result = "True" if is_true else "False"

    lines = [ln.strip() for ln in content.split("\n") if ln.strip()]
    if lines and lines[-1].lower().startswith("판단:"):
        evidence_summary = " ".join(lines[:-1]).strip() if len(lines) > 1 else snippet_text
    else:
        evidence_summary = snippet_text

    return {
        "claim": claim,
        "result": result,
        "evidence": evidence_summary if evidence_summary else "(no evidence)",
        "sources": source_text,
    }

### 4) FactScore 계산
- 모든 Claim 에 대해 True/False 판정을 수행한 후
- FactScore 를 계산한다

In [4]:
# (4) FactScore 계산

def compute_factscore(text: str):
    claims = extract_claims(text)
    if not claims:
        print("추출된 claim이 없습니다.")
        return 0.0, []

    verified = 0
    checked = 0
    details = []

    for c in claims:
        res = verify_claim(c)
        details.append(res)
        print(f"{res['claim']} → {res['result']}")
        print(f"근거: {res['evidence']}")
        print(f"출처:\n{res['sources']}\n")
        if res["result"] == "True":
            verified += 1
        checked += 1

    score = verified / checked if checked else 0.0
    print(f"FactScore = {verified} / {checked} = {score:.2f}")
    return score, details

In [ ]:
# (5) 테스트

text = """
The Earth orbits the Sun.
The Moon is larger than the Earth.
The Eiffel Tower is located in Paris, France.
The Berlin Wall fell in 1989.
"""

score, rows = compute_factscore(text)
print(f"최종 FactScore: {score:.2f}")

The Earth orbits the Sun. → True
근거: 지구는 태양을 평균 1억 4960만 km의 거리에서 공전하며, 이는 과학적으로 입증된 사실입니다.
출처:
- https://en.wikipedia.org/wiki/Earth%27s_orbit
- https://science.nasa.gov/learn/basics-of-space-flight/chapter2-1/
- https://www.astronomy.com/science/when-did-we-realize-that-the-earth-orbits-the-sun/

The Moon is larger than the Earth. → False
근거: 주어진 Claim은 "The Moon is larger than the Earth."입니다. 검색 결과에 따르면, 달은 지구의 약 27% 크기이며, 지구는 달보다 약 4배 더 넓습니다. 따라서 달이 지구보다 크다는 주장은 사실이 아닙니다.
출처:
- https://www.reddit.com/r/Astronomy/comments/14unpd4/is_the_moon_size_to_earth_size_ratio_greater_than/
- https://www.space.com/18135-how-big-is-the-moon.html
- https://science.nasa.gov/solar-system/moon/five-things-to-know-about-the-moon/

The Eiffel Tower is located in Paris, France. → True
근거: 주어진 Claim은 에펠탑이 프랑스 파리에 위치하고 있다는 내용으로, 검색 결과에서도 에펠탑이 파리의 샹 드 마르스에 위치한다고 명확히 언급하고 있습니다.
출처:
- https://en.wikipedia.org/wiki/Eiffel_Tower
- https://www.toureiffel.paris/en/access-map
- https://www.britannica.com/questi

In [6]:
# (6) DataFrame 요약

df = pd.DataFrame(rows)
df[["claim", "result"]]

,claim,result
0,The Earth orbits the Sun.,True
1,The Moon is larger than the Earth.,False
2,"The Eiffel Tower is located in Paris, France.",True
3,The Berlin Wall fell in 1989.,True
